In [ ]:
!pip install contractions

import pandas as pd
import string

import re
import contractions
import nltk


In [ ]:
df = pd.read_csv('messages.csv')
def clean_ling_spam_text(text):
    text = contractions.fix(text)                          # Expand contractions
    text = text.lower()                                    # Convert to lowercase
    text = re.sub(r'http\S+|www\S+', '', text)             # Remove URLs
    text = re.sub(r'\S+@\S+', '', text)                    # Remove email addresses
    text = re.sub(r'\d+', '', text)                        # Remove numbers
    text = re.sub(r'[^\w\s]', '', text)                    # Remove punctuation
    text = re.sub(r'\s+', ' ', text)                       # Normalize spaces
    return text.strip()                                    # Strip leading/trailing spaces

# Apply to dataset
df['text'] = df['subject'].fillna('') + ' ' + df['message'].fillna('')
df['clean_text'] = df['text'].apply(clean_ling_spam_text)


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(df['label'])
X_train, X_test, y_train, y_test = train_test_split( df['clean_text'], Y,  test_size=0.2, random_state=42)


In [ ]:
!pip install tensorflow-text==2.18.0
!pip install tensorflow==2.18.0 --upgrade


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text  # Important! Register custom ops like CaseFoldUTF8

preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3", trainable=False)

# 4. Define function to get BERT embeddings from raw texts
def get_bert_embeddings(texts):
    text_input = tf.constant(texts)
    preprocessed_text = preprocessor(text_input)
    outputs = encoder(preprocessed_text)
    pooled_output = outputs['pooled_output']  # [batch_size, 768]
    return pooled_output

# 5. Get embeddings
train_embeddings = get_bert_embeddings(X_train.tolist())
test_embeddings = get_bert_embeddings(X_test.tolist())

print(train_embeddings.shape)  # e.g., (num_train_samples, 768)
print(type(train_embeddings))  # <class 'tensorflow.python.framework.ops.EagerTensor'>
print(test_embeddings[0])

KeyboardInterrupt: 

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text  # This is needed even if you don't use it directly

# Load BERT preprocessing and encoder
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3", trainable=False)

# Define function to get BERT embeddings
def get_bert_embeddings(texts):
    text_input = tf.constant(texts)
    #print(text_input)
    preprocessed_text = preprocessor(text_input)
    #print(preprocessed_text)
    outputs = encoder(preprocessed_text)
    print(outputs)
    pooled_output = outputs['pooled_output']
    #print(pooled_output)
    return pooled_output

# Get BERT embeddings for training and test sets
train_embeddings = get_bert_embeddings(X_train)
test_embeddings = get_bert_embeddings(X_test)

print(train_embeddings.shape)
print(type(train_embeddings))
print(test_embeddings[0])

{'pooled_output': <tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.80938184, -0.16546664,  0.68999904, ...,  0.44613266,
        -0.5084402 ,  0.81900126],
       [-0.8157158 , -0.30196622, -0.70863587, ..., -0.771869  ,
        -0.61023116,  0.7881911 ]], dtype=float32)>, 'sequence_output': <tf.Tensor: shape=(2, 128, 768), dtype=float32, numpy=
array([[[-0.02621197,  0.19221139,  0.03323635, ..., -0.13233647,
          0.07647157,  0.24538027],
        [-0.5656332 , -0.25131604,  0.31146434, ..., -0.7792762 ,
          0.22006303,  0.59504753],
        [-0.23581746, -0.31642663,  0.6927501 , ..., -0.20723024,
          0.10499597,  0.99023205],
        ...,
        [ 0.11773557,  0.10725212,  0.5276053 , ..., -0.01576325,
          0.0379568 ,  0.3579131 ],
        [ 0.13846585,  0.04988021,  0.5572197 , ...,  0.03026925,
         -0.01479577,  0.276532  ],
        [ 0.05706859, -0.05958192,  0.53499687, ...,  0.07162747,
         -0.04726911,  0.21381557]],

       [[-0.0